<a href="https://colab.research.google.com/github/nakamura196/ndl_ocr/blob/main/ndl_ocr_folder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NDLOCRの実行例：Google Driveを用いた画像の入力と結果の保存

<br/>

---

<br/>

**本ノートブックの改良版であるVersion 2を作成しました。以下も参考にしてください。**

https://colab.research.google.com/github/nakamura196/ndl_ocr/blob/main/ndl_ocr_v2.ipynb

<br/>

---

<br/>


Google Drive上のフォルダを指定して、当該フォルダに含まれる複数画像に対してOCR処理を実行し、指定したフォルダに認識結果を出力します。

## 参考記事

https://zenn.dev/nakamura196/articles/a8227f4524570c

## 参考にしたノートブック

@blue0620 さんが作成したノートブック

https://github.com/blue0620/NDLOCR-GoogleColabVersion/blob/main/NDLOCR_googlecolabversion.ipynb

## 使用方法

- 「1.初期セットアップ」を実行してください。初回のみ実行が必要です。

- 「2. 設定」を変更してください。

- 「3. 実行」を行います。

## 1.初期セットアップ

時間がかかります。初回のみ実行が必要です。

In [ ]:
# @title

%cd "/content/"

# 6. google driveのマウント
from google.colab import drive
drive.mount('/content/drive/')

from IPython.display import clear_output 

# 1. NDLOCRのリポジトリをcloneする(--recursiveを忘れずに！)
!git clone -q --recursive https://github.com/ndl-lab/ndlocr_cli
clear_output()

# 2. 必要なパッケージをインストールする
PROJECT_DIR="/content/ndlocr_cli"
!pip install -q -r {PROJECT_DIR}/requirements.txt
!pip install -q torch==1.8.1+cu111 torchvision==0.9.1+cu111 -f https://download.pytorch.org/whl/lts/1.8/torch_lts.html
clear_output()
!pip install -q mmcv-full==1.4.0 -f https://download.openmmlab.com/mmcv/dist/cu111/torch1.8.0/index.html
##numpyのバージョン問題でcolabでは動かなかったのでアップデートする(参考:https://stackoverflow.com/questions/66060487/valueerror-numpy-ndarray-size-changed-may-indicate-binary-incompatibility-exp)
!pip install -q --upgrade numpy
clear_output()

%cd {PROJECT_DIR}/src/ndl_layout/mmdetection
!python setup.py bdist_wheel
clear_output()
!pip install -q dist/*.whl
clear_output()

# # 4. OCRに必要な学習済みモデルをダウンロードする

%cd {PROJECT_DIR}
!wget -q https://lab.ndl.go.jp/dataset/ndlocr/text_recognition/mojilist_NDL.txt -P ./src/text_recognition/models
!wget -q https://lab.ndl.go.jp/dataset/ndlocr/text_recognition/ndlenfixed64-mj0-synth1.pth -P ./src/text_recognition/models
!wget -q https://lab.ndl.go.jp/dataset/ndlocr/ndl_layout/ndl_layout_config.py -P ./src/ndl_layout/models
!wget -q https://lab.ndl.go.jp/dataset/ndlocr/ndl_layout/epoch_140_all_eql_bt.pth -P ./src/ndl_layout/models
!wget -q https://lab.ndl.go.jp/dataset/ndlocr/separate_pages_ssd/weights.hdf5 -P ./src/separate_pages_ssd/ssd_tools
clear_output()

# 5. 環境変数を追加する
import os
os.environ["PYTHONPATH"]=os.environ["PYTHONPATH"]+":"+f"{PROJECT_DIR}/src/text_recognition/deep-text-recognition-benchmark"

clear_output()

from pathlib import Path
import glob
from tqdm import tqdm

## 2. 設定

OCRの適用対象のファイルを設定します。

- input_dir: 画像が格納されたフォルダへのパス
- output_dir: 出力するフォルダへのパス
- input_structure: f または s （入力ディレクトリの形式）
  - 詳細はこちらをご確認ください。 https://github.com/ndl-lab/ndlocr_cli#%E5%85%A5%E5%87%BA%E5%8A%9B%E4%BB%95%E6%A7%98
- extensions: 実行対象のファイル拡張子（,区切りで複数指定可）

In [ ]:
#@title
input_dir  = "/content/drive/MyDrive/ndl_ocr/input/" #@param {type:"string"}
output_dir  = "/content/drive/MyDrive/ndl_ocr/output/" #@param {type:"string"}

input_structure = "s" #@param ["f", "s"]
extensions = "jpeg" #@param {type:"raw"}

process = "\u3059\u3079\u3066: \u30CE\u30C9\u5143\u5206\u5272,\u50BE\u304D\u88DC\u6B63,\u30EC\u30A4\u30A2\u30A6\u30C8\u62BD\u51FA,\u6587\u5B57\u8A8D\u8B58(OCR)" #@param ["\u3059\u3079\u3066: \u30CE\u30C9\u5143\u5206\u5272,\u50BE\u304D\u88DC\u6B63,\u30EC\u30A4\u30A2\u30A6\u30C8\u62BD\u51FA,\u6587\u5B57\u8A8D\u8B58(OCR)", "\u50BE\u304D\u88DC\u6B63,\u30EC\u30A4\u30A2\u30A6\u30C8\u62BD\u51FA,\u6587\u5B57\u8A8D\u8B58(OCR)", "\u30EC\u30A4\u30A2\u30A6\u30C8\u62BD\u51FA,\u6587\u5B57\u8A8D\u8B58(OCR)", "\u6587\u5B57\u8A8D\u8B58(OCR)"]
skip_if_output_dir_exists = False #@param {type:"boolean"}

# 末尾の/（スラッシュ）の有無を無視するように修正
output_dir = str(Path(output_dir))
input_dir = str(Path(input_dir))

## 3. 実行

In [ ]:
#@title

def createParentDir(path):
  basename = os.path.basename(path)
  parent_path = os.path.dirname(path)
  os.makedirs(parent_path, exist_ok=True)

def listFiles(input_dir, exts):
  paths = []
  ext_list = exts.split(",")
  for ext in ext_list:
    files = glob.glob("{}/**/*.{}".format(input_dir, ext), recursive=True)
    paths.extend(files)

  return sorted(paths)

files = listFiles(input_dir, extensions)

p = "0..3"
if process == "傾き補正,レイアウト抽出,文字認識(OCR)":
  p = "1..3"
elif process == "レイアウト抽出,文字認識(OCR)":
  p = "2..3"
elif process == "文字認識(OCR)":
  p = "3"

# Sigle input dir mode の場合
if input_structure == "s":
  exec_flg = True
  if os.path.exists(output_dir):
      # スキップフラグがTrueならContinue
      if skip_if_output_dir_exists:
        exec_flg = False
  if exec_flg:
    !python main.py infer -s $input_structure $input_dir $output_dir -x -i -p $p
  else:
    print("######## 出力フォルダが既に存在しているため、処理を終了します。 ########")
else:
  # Image file mode の場合
  for file in tqdm(files):
    # 出力パスの作成
    output_path = "{}/{}_p{}".format(output_dir, file.replace(input_dir, ""), p)

    if os.path.exists(output_path):
      # スキップフラグがTrueならContinue
      if skip_if_output_dir_exists:
        continue
      !rm -rf $output_path

    createParentDir(output_path)
    !python main.py infer -s f $file $output_path -x -i -p $p
    clear_output()

    # 以下、出力ファイルの整理
    basename_without_ext = os.path.splitext(os.path.basename(file))[0]

    input_tmp_path = "{}/{}".format(output_path, basename_without_ext)
    !mv $input_tmp_path/* $output_path

    !rm $output_path/opt.json
    !rm -rf $output_path/$basename_without_ext